In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as wb
assets=['PETR4.SA','ABEV3.SA','CIEL3.SA','ITUB4.SA']
portfolio=pd.DataFrame()



In [31]:
import datetime
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2019, 6, 15)

In [46]:
for t in assets:   
    portfolio[t]=wb.DataReader(t,data_source='yahoo',start='2012-1-1',end='2019-6-20')['Adj Close']

sec_ret=np.log(portfolio/portfolio.shift(1))


,PETR4.SA,ABEV3.SA,CIEL3.SA,ITUB4.SA
Date,,,,
2012-01-02,NaN,NaN,NaN,NaN
2012-01-03,0.030813,-0.011461,0.003716,0.024669
2012-01-04,0.007114,-0.023524,0.003497,0.001145
2012-01-05,-0.011587,-0.019992,-0.014478,-0.007468
2012-01-06,0.000896,-0.002530,-0.006270,0.007468
2012-01-09,0.013345,-0.001950,0.003975,0.006282
2012-01-10,0.011861,-0.005079,0.005414,0.013008
2012-01-11,0.000436,0.000000,0.010742,0.001406
2012-01-12,0.007393,0.004493,-0.002468,0.002240


In [47]:
sec_ret.tail()

,PETR4.SA,ABEV3.SA,CIEL3.SA,ITUB4.SA
Date,,,,
2016-12-23,0.016284,0.011314,-0.008903,0.035329
2016-12-26,0.012561,0.011187,0.001784,0.007974
2016-12-27,-0.000694,-0.004336,-0.021662,-0.000609
2016-12-28,0.025352,0.007421,0.005825,0.023266
2016-12-29,0.006071,0.010420,0.011903,0.010992


## Matriz de covariância

In [48]:
# Vamos assumir 250 dias ao ano engociados.
covariance=sec_ret.cov()*250
covariance


,PETR4.SA,ABEV3.SA,CIEL3.SA,ITUB4.SA
PETR4.SA,0.258327,0.035671,0.047923,0.094803
ABEV3.SA,0.035671,0.056982,0.020953,0.029629
CIEL3.SA,0.047923,0.020953,0.089176,0.035104
ITUB4.SA,0.094803,0.029629,0.035104,0.101497


In [49]:
marketcov=covariance.iloc[0,1]
marketcov


0.03567084329641455

In [51]:
market_var=sec_ret['ITUB4.SA'].var()*250
market_var

0.1014966114698867

In [52]:
market_beta=marketcov/market_var
market_beta

0.35144861271548783

In [9]:
# Expected Return
Exp_ret= 0.025+market_beta*0.05
Exp_ret

0.055797882934685655

In [54]:
# Calculation os Sharpe ratio
sharpe=(Exp_ret-0.025)/(sec_ret['ITUB4.SA'].std()*250**0.5)
sharpe

0.09667075094022505